In [2]:
import jetson.inference, jetson.utils
import cv2

In [3]:
jetson.inference.VERSION

'1.0.0'

In [4]:
jetson.utils.VERSION

'1.0.0'

In [5]:
print(dir(jetson.utils))

['VERSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'adaptFontSize', 'cudaAllocMapped', 'cudaConvertColor', 'cudaCrop', 'cudaDeviceSynchronize', 'cudaDrawCircle', 'cudaDrawLine', 'cudaDrawRect', 'cudaFont', 'cudaFromNumpy', 'cudaImage', 'cudaMalloc', 'cudaMemcpy', 'cudaMemory', 'cudaNormalize', 'cudaOverlay', 'cudaResize', 'cudaToNumpy', 'glDisplay', 'gstCamera', 'loadImage', 'loadImageRGBA', 'logUsage', 'saveImage', 'saveImageRGBA', 'videoOutput', 'videoSource']


In [6]:
print(dir(jetson.utils.videoSource))

['Capture', 'Close', 'GetFrameRate', 'GetHeight', 'GetWidth', 'IsStreaming', 'Open', 'Usage', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [7]:
#cap = jetson.utils.videoSource("csi://0")  # MIPI CSI camera #0
cap = jetson.utils.videoSource("/dev/video2")  # V4L2 camera #0
#cap = jetson.utils.videoSource("nemo_video.mp4") 

In [8]:
print(cap.Usage())

videoSource arguments: 
    input_URI            resource URI of the input stream, for example:
                             * /dev/video0              (V4L2 camera #0)
                             * csi://0                  (MIPI CSI camera #0)
                             * rtp://@:1234             (RTP stream)
                             * rtsp://user:pass@ip:1234 (RTSP stream)
                             * file://my_image.jpg      (image file)
                             * file://my_video.mp4      (video file)
                             * file://my_directory/     (directory of images)
  --input-width=WIDTH    explicitly request a width of the stream (optional)
  --input-height=HEIGHT  explicitly request a height of the stream (optional)
  --input-rate=RATE      explicitly request a framerate of the stream (optional)
  --input-codec=CODEC    RTP requires the codec to be set, one of these:
                             * h264, h265
                             * vp8, vp9
        

In [9]:
cap.IsStreaming()

False

In [10]:
cap.Open()

In [11]:
cap.IsStreaming()

True

In [12]:
cap.GetFrameRate()

30

In [16]:
w,h = cap.GetWidth(), cap.GetHeight()

w,h

(1024, 768)

In [14]:
img = cap.Capture()

In [15]:
type(img)

jetson.utils.cudaImage

In [28]:
print(dir(img))

['__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'channels', 'format', 'freeOnDelete', 'height', 'mapped', 'ptr', 'shape', 'size', 'width']


In [36]:
img.height, img.width, img.channels, img.shape, img.size, img.ptr, img.format, 

(768, 1024, 3, (768, 1024, 3), 2359296, 4315086848, 'rgb8')

In [37]:
img.mapped

True

In [40]:
cuImg = cv2.cuda_GpuMat()
cuImg.create((w, h), cv2.CV_8UC3)

In [41]:
cuImg.upload(jetson.utils.cudaToNumpy(img))

In [45]:
cuImg.size(), cuImg.channels()

((1024, 768), 3)

In [46]:
cuGray = cv2.cuda_GpuMat()
cuGray.create((w, h), cv2.CV_8UC1)

In [ ]:

while cap.IsStreaming :
    cuImg.upload(jetson.utils.cudaToNumpy(cap.Capture()))

    cv2.cuda.cvtColor(cuImg, cv2.COLOR_BGR2GRAY, cuGray)

    #gray = cuGray.download()
    cv2.imshow("Stream Output", cuGray)

    if cv2.waitKey(1) == ord("q"):
        break

cap.Close()
cv2.destroyAllWindows()